In [ ]:
import sklearn
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# data import

In [ ]:
path = "/content/drive/Shareddrives/Multicampus/Preprocessing/"
dummy = pd.read_csv(path+"0414_dummy(최종).csv")
pivot12_under = pd.read_csv(path+"pivot12_mnmx_under.csv")
pivot12_under_ind = pd.read_csv(path+"pivot12_mnmx_under_ind.csv")

In [ ]:
dummy.drop(['new_code','AGEG1','AGEG2'],inplace=True, axis=1)
# AGEG1,2 어떻게 처리된 건지 알아보기

In [ ]:
# u_dummy_pivot12 = pd.merge(dummy, pivot12_under, how='inner', on='SUBJECT_ID')
u_dummy_pivot12_ind = pd.merge(dummy, pivot12_under_ind, how='inner', on='SUBJECT_ID')
# u_ind = pd.concat([u_dummy_pivot12_ind['SUBJECT_ID'],u_dummy_pivot12_ind.iloc[:,105:]], axis=1)
# u_dummy = pd.merge(dummy, pivot12_under['SUBJECT_ID'], how='right', on='SUBJECT_ID')
# u_dummy_ind = pd.merge(dummy, u_ind, how='right', on = 'SUBJECT_ID')
# u_dummy_ind.info()

# package import

In [ ]:
# pipeline
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV

# imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer

# scaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# feature_selection
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier

# model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# result
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# mean impute, 정규화, train-test split

In [ ]:
def mean_z(data, target, strategy='mean'):
    imputer = SimpleImputer(strategy=strategy)
    imputed = imputer.fit_transform(data)
    df_imputed = pd.DataFrame(imputed, columns=data.columns)
        
    X = df_imputed
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 표준화
    sc = StandardScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
def mean_n(data, target, strategy='mean'):
    imputer = SimpleImputer(strategy=strategy)
    imputed = imputer.fit_transform(data)
    df_imputed = pd.DataFrame(imputed, columns=data.columns)
        
    X = df_imputed
    y = target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # 정규화
    sc = MinMaxScaler()
    sc.fit(X_train)
    X_train = sc.transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, y_train, y_test

In [ ]:
inter_idx = [0, 6, 7, 10, 12, 16, 18, 19, 60, 61, 62, 65, 66, 68, 78, 80, 81, 82, 83, 86, 87, 91, 101, 102, 123]
inter_df = u_dummy_pivot12_ind.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','staytime'],axis=1).iloc[:,inter_idx]

union_idx = [0, 6, 7, 10, 12, 13, 16, 18, 19, 60, 61, 62, 64, 65, 66, 68, 70, 71, 77, 78, 80, 81, 82, 83, 85, 86, 87, 89, 91, 92, 98, 99, 101, 102, 123]
union_df = u_dummy_pivot12_ind.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','staytime'],axis=1).iloc[:,union_idx]

mdi_idx = [ 16,   0,  92,  71,  82,  60,  86,  18,  61,  65,  81,  62,  89,
             19,   6,  87,  77,   7, 123,  83, 102,  98, 101,  80,  10,  78,
             68,  66,  91,  12]
mdi_df = u_dummy_pivot12_ind.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','staytime'],axis=1).iloc[:,mdi_idx]

perm_idx = [ 16,   0,  60, 123,   6,   7,  87, 102,  82,  62,  80,  10, 101,
             81,  61,  91,  19,  66,  18,  12,  83,  78,  85,  70,  13,  64,
             65,  99,  86,  68]
perm_idx = u_dummy_pivot12_ind.drop(['SUBJECT_ID','HOSPITAL_EXPIRE_FLAG','staytime'],axis=1).iloc[:,perm_idx]

In [ ]:
inter_df.shape

(8942, 25)

In [ ]:
mean_n_inter = mean_n(inter_df, u_dummy_pivot12_ind.HOSPITAL_EXPIRE_FLAG)

NameError: ignored

In [ ]:
X_train, X_test, y_train, y_test = mean_n_inter

# GridSearchCV

## scores 정의

In [ ]:
scores = {'AUC': 'roc_auc', 
           'Accuracy': make_scorer(accuracy_score), 
           'precision':make_scorer(precision_score, average='micro'), 
           'recall': make_scorer(recall_score, average='micro'),
           'f1-score':make_scorer(f1_score, average='micro')}

## SVM

In [ ]:
param_grid = [
              # {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [10, 100, 1000], 'gamma': [0.01, 0.001], 'kernel': ['rbf']},              
              # {'C': [1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001], 'kernel': ['sigmoid']}, # 별로임
              ]

grid = GridSearchCV(estimator = SVC(cache_size = 500, random_state = 42), 
                    param_grid = param_grid, 
                    scoring = scores,
                    refit='f1-score', 
                    n_jobs = -1,
                    cv = 5)  # refit: fit 기준 

grid.fit(X_train, y_train)
# https://wikidocs.net/16599

GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=500,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C...
                         {'C': [1, 10, 100, 1000],
                          'gamma': [0.01, 0.001, 0.0001],
                          'kernel': ['sigmoid']}],
             pre_dispatch='2*n_jobs', refit='f1-score',
             return_train_score=False,
             scoring={'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score),
                      'f1-score': make_scorer(f1_score, average=micro),
             

In [ ]:
print(grid.best_params_)
print(grid.best_score_)
print(grid.best_estimator_)

{'C': 1000, 'gamma': 0.01, 'kernel': 'rbf'}
0.7517190081800373
SVC(C=1000, break_ties=False, cache_size=500, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)


In [ ]:
svm = pd.DataFrame(grid.cv_results_).sort_values(by=['rank_test_f1-score'], ascending=True)
svm[['param_C','param_kernel','param_gamma', 
    'mean_test_AUC', 'mean_test_Accuracy', 'mean_test_precision',
    'mean_test_recall', 'mean_test_f1-score']].head()                        

param_C param_kernel  ... mean_test_recall  mean_test_f1-score
13    1000          rbf  ...         0.751719            0.751719
10     100          rbf  ...         0.748204            0.748204
3     1000       linear  ...         0.738619            0.738619
2      100       linear  ...         0.737980            0.737980
14    1000          rbf  ...         0.737181            0.737181

[5 rows x 8 columns]

In [ ]:
estimator = grid.best_estimator_
y_pred = estimator.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('-'*20)
print('model:',estimator)
print('오차 행렬: ')
print(confusion)
print('AUC: {4: .4f}, 정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1 socre: {3:.4f}'.format(accuracy , precision ,recall, f1, roc_auc))
print('-'*20)

--------------------
model: SVC(C=1000, break_ties=False, cache_size=500, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)
오차 행렬: 
[[1273  304]
 [ 345  761]]
AUC:  0.7476, 정확도: 0.7581, 정밀도: 0.7146, 재현율: 0.6881, F1 socre: 0.7011
--------------------


# 

In [ ]:
mean_n_union = mean_n(union_df, u_dummy_pivot12_ind.HOSPITAL_EXPIRE_FLAG)
X_train, X_test, y_train, y_test = mean_n_inter

## Logistic Regression

In [ ]:
param_grid = [
              # {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
              {'C': [10, 100, 1000], 'gamma': [0.01, 0.001], 'kernel': ['rbf']},              

              {'penalty':['l2'], 'solver':[]
                  
              }
              ]

grid = GridSearchCV(estimator = LogisticRegression(random_state = 42), 
                    param_grid = param_grid, 
                    scoring = scores,
                    refit='f1-score', 
                    n_jobs = -1,
                    cv = 5)  # refit: fit 기준 

grid.fit(X_train, y_train)
# https://wikidocs.net/16599

In [ ]:
lr = LogisticRegression(penalty=12, solver = sag, C=1.0, )

In [ ]:
models = [GradientBoostingRegressor(random_state = 0), 
          LinearRegression(),
          KNeighborsRegressor(),
          RandomForestRegressor(random_state = 0),
          SVR()]

# 이전과정에서 r2값이 음수였던 모델은 제외한 경우
# models = [GradientBoostingRegressor(random_state = 0), 
#           LinearRegression(),
#           KNeighborsRegressor(),
#           RandomForestRegressor(random_state = 0),
#           SVR()]

results_r2 = {}
results_MAE = {}
results_RMSE = {}

for model in models:
    
    # 모델 정의 및 모델 형성
    reg_model = model
    reg_model.fit(X_train, y_train)
    
    # 예측
    y_pred = reg_model.predict(X_test)
    
    # 모델 이름
    name = str(model).split("(")[0]

    results_r2[name] = r2_score(y_test, y_pred)
    results_MAE[name] = metrics.mean_absolute_error(y_test, y_pred)
    results_RMSE[name] = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print('{} done.'.format(name))
# R2 score results
fig, ax = plt.subplots()
ind = range(len(results_r2))
ax.barh(ind, list(results_r2.values()), align='center', 
        color = '#55a868', alpha=0.8)
ax.set_yticks(ind)
ax.set_yticklabels(results_r2.keys())
ax.set_xlabel('R-squared score')
ax.tick_params(left=False, top=False, right=False) 
ax.set_title('Comparison of Regression Models')
fig.savefig(path+'dummydata_compare_models.png', bbox_inches = 'tight')